In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import random 
import json 
import math

In [4]:
from lib.local_database import Database, Collection, Question, db

from lib.html_viewer import view_questions as view
from lib.html_viewer import view_components as view_comp
from lib.html_viewer import extract_vectorimage as extract
from lib.html_viewer import svg_to_component as compose
from lib.html_viewer import tabl,trow,telm,underline,inpt,mini_red
from lib.html_viewer import h3,p,empty,span,div,mkrow,mktabl

from lib.svg_generator import diagram

In [5]:
from lib.svg_generator import *

## setup

In [44]:
winkel_tip = h3("beim Gegenwinkel sind alpha und beta gleich groß:")

winkel_tip += f'''<div style="background:white;display:inline">  {svg(
        
        line((900,500),(100,500),),
        line((800,800),(200,200),),

        line(*bow((500,500),200,270,315)),
        line(*bow((500,500),200,90,135)),



        text(400,470,"α"),
        text(600,550,"β"),

        text(150,900,"α = β ")

        
        )}</div>'''


winkel_tip += h3("beim Nebenwinkel  ergeben α und β zusammen 180° :")

winkel_tip += f'''<div style="background:white;display:inline">  {svg(
        
        line((900,500),(100,500),),
        line((800,800),(200,200),),

        line(*bow((500,500),200,270,315)),
        line(*bow((500,500),120,315,90)),



        text(400,470,"α"),
        text(500,450,"β"),

        text(150,900,"α + β = 180°"),
        text(150,940,"α = 180° - β")


        
        )}</div>'''




view_comp(winkel_tip)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [45]:
collection:Collection = db.get_collection("math","msa")

In [46]:
view_comp(svg(line((0,0),(500,500))))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [8]:

def comp(deg:float):
    html = f'''<div style="background:white;display:inline">  {svg(
        
        line(
            (800,700),
            (250,200),
            (150,500),
            (850,500)),

        line(*bow((580,500),200,270,310)),
        line(*bow((580,500),200,90,130)),
        line(*bow((250,200),140,135,195)),


        text(450,470,"α"),
        text(650,550,f"β={deg}°"),
        text(230,300,"γ=63°"),

        text(150,900,"Zeichnung ist nicht Maßstabsgetreu")
        
        )}</div>'''

    return html

view_comp(comp(30))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [9]:
def gen(deg:float):
    c = comp (deg)
    c += p("wie groß ist beta?")
    return Question.standart(c,deg,[],tip=winkel_tip)

view(gen(40))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [10]:
q = [gen(d) for d in range(20,50,3)]

In [11]:
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


### anteil

In [12]:
def comp(pieces=[]):

    rot = random.randint(0,360)

    html = f'''<div style="background:white;display:inline">  {svg(
        
        line(*bow((500,500),300,0,360)),
        line((500,500,),*bow((500,500),300,0   + rot, 60  + rot),(500,500),fill = "grey" if 60  in pieces else "white"),
        line((500,500,),*bow((500,500),300,60  + rot, 180 + rot),(500,500),fill = "grey" if 120 in pieces else "white"),
        line((500,500,),*bow((500,500),300,240 + rot, 270 + rot),(500,500),fill = "grey" if 30 in pieces else "white"),
        line((500,500,),*bow((500,500),300,270 + rot, 360 + rot),(500,500),fill = "grey" if 90 in pieces else "white"),

        text(150,900,"Zeichnung ist nicht Maßstabsgetreu"),
        
        )}</div>'''

    return html

view_comp(comp([120,90]))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [13]:
def gen(pieces = []):

    possibs = ["1/2","1/3","1/4","1/6","1/12"]

    corrects=  list(map(lambda p: f"1/{int(360/p)}",pieces))

    for x in corrects:
        assert x in possibs ,f"illegal option {x}"

    wrongs = list(filter(lambda p:p not in corrects,possibs))

    return Question.multiple_choice(comp(pieces)+p("welcher teil ist markiert?"),corrects,wrongs,"")

view(gen([120,30]))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [14]:
q = [
    gen([30,120]),
    gen([120,90]),
    gen([90,60]),
    gen([60,30]),
]

In [15]:
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [16]:
collection.add_series(q,5,"anteile erkennen")

### potenzen vergleichen 

In [17]:
def gen(*item):

    nums = map (lambda i:i[0]**i[1],item)

    mx = max(nums)
    wrongs = list(filter( lambda x: x[0]**x[1] != mx, item))



    corrects = list(filter( lambda x: x[0]**x[1] == mx, item))

    wrongs = list(map(lambda x: f"{x[0]}^{x[1]}",wrongs))
    corrects = list(map(lambda x: f"{x[0]}^{x[1]}",corrects))

    return Question.multiple_choice(h3("Welche Potenz hat das größere Ergebniss?"),corrects,wrongs,"")

view(gen((1,3),(4,1)))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [18]:
q = [
    gen((4,3),(8,4),(2,8)),
    gen((4,2),(6,3),(2,6)),
    gen((4,3),(100,0),(2,8)),
    gen((3,2),(20,1),(2,4)),
    gen((4,3),(50,2),(2,5)),
]
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [19]:
collection.add_series(q,4,"potenzen vergleichen")

### simple Fläche berechnen

### simple wahrscheinlichkeit

In [100]:
def comp(bnum,p1,p2):

    balls = []

    x = 210
    step = 110
    for i in range(bnum):
        balls.append(line(*bow((x+step*i,340),50,0,-1),fill='black'))
    
    balls.append(line(*bow((x+step*0.5,240),50,0,-1)))


    return h3('''In einem Gefäß befinden sich schwarze und weiße Kugeln.
Das Baumdiagramm zeigt an, mit welcher Wahrscheinlichkeit eine schwarze oder
weiße Kugel gezogen werden kann.''')+h3("wie viele Weiße Kugeln müssen in der Urne enthalten sein?")+svg(
        line((850,100),(850,400),(150,400),(150,100),),
        *balls,

        line((250,750),(700,650)),
        line((250,750),(700,850)),
        line(*bow((700,650),50,0,-1),fill="white"),
        line(*bow((700,850),50,0,-1),fill="black"),
        text(500,650,p1),
        text(500,870,p2),
    )


view_comp(comp(3,"p1","p2"))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [109]:
q=[
    Question.standart(comp(4,"1/2","1/2"),4,[],),
    Question.standart(comp(3,"1/4","3/4"),1,[],),
    Question.standart(comp(4,"3/4","1/4"),12,[],),
    Question.standart(comp(1,"3/4","1/4"),4,[],),
    Question.standart(comp(2,"5/6","1/6"),10,[],),
]
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


### Statistik Spannweite

In [153]:
def gen():

    mn = random.random()*10
    mx = mn + random.random()*10

    nums = [round(random.random(),1) for _ in range(4)]
    nums += [round(mn,1),round(mx,1)]

    span = max(nums) - min(nums)
    span = round(span,1)

    nums = list(map(str,nums))

    nums = random.sample(nums,len(nums))

    tit = h3("was ist die Spannweite der folgenden Daten")
    tit += p(" ".join(nums))

    return Question.standart(tit,str(span))

view(gen())

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [154]:
q = [gen() for _ in range(10)]

In [156]:
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [155]:
collection.add_series(1,4,"statistik spannweite")

### wissenschaftliche_schreibweise

In [40]:
tip = p("bei der Wissenschaftlichen Schreibweise schreibt man erst eine zahl zwischen 1 und 10 auf.")
tip += p("dann schreibt man mal 10^n wobei n angibt wie viele stellen die Zahl verschoben werden muss.")
tip += p("wenn n positiv ist muss die zahl vergrößert werden und wenn n negativ ist muss sie verkleinert werden.")

tip += p("2.1 • 10^2 = 210")
tip += p("2.1 • 10^1 = 21")
tip += p("2.1 • 10^-1 = 0.21")
tip += p("2.1 • 10^-2 = 0.021")


In [41]:
exps = [5,4,3,2,-2,-3,-4,-5]
def gen():
    mant = random.random() * 10 
    mant = round(mant,1)

    exp = random.choice(exps)

    res = mant * int(10 ** exp)

    tit = h3("geben Sie die Dezimalzahl an.")
    tit += p(f"{mant} • 10^{exp}")

    return Question.standart(tit,res,tip = tip)

gen()

<Q:  <h3>geben Sie die Dezimalzahl an.</h3><p : [0.0]>

In [42]:
q = [gen() for _ in range(10)]
view(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [48]:
collection.add_series(q,4,"Wissenschaftliche Schreibweise lesen")

### lineare nullstelle

In [ ]:
svg()

In [231]:
class CoordinateSystem:

    def __init__(self,xrange = [-5,5],yrange = [-5,5]):
        self.xrange, self.yrange = xrange, yrange

        self.xspan = max(xrange) - min(xrange)
        self.yspan = max(yrange) - min(yrange)

        minspan = min(self.xspan,self.yspan)

        arrowheadsize = minspan * 0.03

        self.items = []

        for i in range(xrange[0]+1,xrange[1]):
            self.line((i,yrange[0]),(i,yrange[1]),color="#bb9")
            self.line((i,0+arrowheadsize*0.5),(i,-arrowheadsize*0.5))

        for i in range(xrange[0]+1,xrange[1]):
            self.line((yrange[0],i),(yrange[1],i),color="#bb9")
            self.line((arrowheadsize*0.5,i),(-arrowheadsize*0.5,i))


        self.line((self.xrange[0],0),(self.xrange[1],0))
        self.line((self.xrange[1]-arrowheadsize,0+arrowheadsize),(self.xrange[1],0),(self.xrange[1]-arrowheadsize,0-arrowheadsize))

        self.line((0,self.yrange[0]),(0,self.yrange[1]))
        self.line((0+arrowheadsize,self.yrange[0]+arrowheadsize),(0,self.yrange[0]),(0-arrowheadsize,self.yrange[0]+arrowheadsize))
    

    def wpos(self,p):
        return ((p[0]-self.xrange[0])*1000/self.xspan,(p[1]-self.yrange[0])/self.yspan*1000)

    def line(self,*points,color="black",fill="none"):
        points = map(self.wpos ,points)
        self.items.append(line(*points,color=color,fill=fill))

cs = CoordinateSystem()

view_comp(svg(*cs.items))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [167]:
class CoordinateSystem:
    def __init__(self,xrange = [-5,5],yrange = [-5,5]):
        self.xrange, self.yrange = xrange, yrange

        xspan = max(xrange) - min(xrange)
        yspan = max(yrange) - min(yrange)

        minspan = min(xspan,yspan)

        self.grid = Grid(xspan,yspan)

        arrowheadsize = minspan * 0.03

        self.grid.add_figure((0,-yrange[0]),(xspan,-yrange[0]))
        self.grid.add_figure((xspan-arrowheadsize,-yrange[0]+arrowheadsize),(xspan,-yrange[0]),(xspan-arrowheadsize,-yrange[0]-arrowheadsize),fill="none")

        self.grid.add_figure((-xrange[0],0),(-xrange[0],yspan))
        self.grid.add_figure((-xrange[0]-arrowheadsize,0+arrowheadsize),(-xrange[0],0),(-xrange[0]+arrowheadsize,0+arrowheadsize),fill="none")


        for i in range(xrange[0]+1,xrange[1]):

            self.add_figure((i,.5*arrowheadsize),(i,-.5*arrowheadsize))

        for i in range(yrange[0]+1,yrange[1]):
            self.add_figure((.5*arrowheadsize,i),(-.5*arrowheadsize,i))

        self.get_html = self.grid.get_html

    def wpos(self,p):
        return (p[0]-self.xrange[0],p[1]-self.yrange[0])

    def add_figure(self,*points,color="black",fill="none"):
        points = map(self.wpos ,points)
        self.grid.add_figure(*points,color=color,fill=fill)


In [ ]:
diagram

In [168]:
cs = CoordinateSystem()


view_comp(cs.get_html())

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [145]:

cs = CoordinateSystem(xrange=[-4,5])

view_comp(cs.get_html())

[(0, 5.135), (0, 4.865)]
[(1, 5.135), (1, 4.865)]
[(2, 5.135), (2, 4.865)]
[(3, 5.135), (3, 4.865)]
[(4, 5.135), (4, 4.865)]
[(5, 5.135), (5, 4.865)]
[(6, 5.135), (6, 4.865)]
[(7, 5.135), (7, 4.865)]
[(8, 5.135), (8, 4.865)]
view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


## save

In [77]:
collection.save()

sname: gegenwinkel rstep: 0.5
